In [1]:
import numpy as np
import pandas as pd 
import jieba 
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LogisticRegression
from collections import Counter
from math import isnan
from sklearn.feature_extraction.text import TfidfTransformer
import copy

c:\python3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
txt = pd.read_excel('数据比较好的.xlsx')
Y = txt['标签']

In [3]:
jieba.load_userdict(Y)
stopwords = [line.strip() for line in open('../input/stopword.txt', 'r', encoding='utf-8').readlines()] 

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\hebo\AppData\Local\Temp\jieba.cache
Loading model cost 0.798 seconds.
Prefix dict has been built succesfully.


In [4]:
list_des = txt['商品描述'].tolist()
X = []
with open('分词拿去做word2vect.txtct.txt','w',encoding = 'utf8') as file:
    for i in range(len(list_des)):
        des = list_des[i].replace(' ','')
        seg_list = jieba.lcut(des)
        word_list = [] 
        for seg in seg_list:
            if seg not in stopwords:
                word_list.append(seg)
        X.append((' ').join(word_list))
        file.write((' ').join(word_list) +'\n')

In [5]:
txt['商品描述'] = pd.Series(X)

In [6]:
frame = pd.DataFrame(columns=['商品描述','标签'])
dic = Counter(Y)
for i in dic.keys():
    data = txt.loc[txt['标签'] == i]
    if dic[i] > 200:
        frame = pd.concat([frame,data[:200]],axis = 0)
        continue
    else:
        frame = pd.concat([frame,txt.loc[txt['标签'] == i]],axis = 0)

In [7]:
frame = frame.reset_index(drop=True)
frame.shape

(61467, 2)

In [8]:
vectorizer = CountVectorizer()
fre_matrix = vectorizer.fit_transform(frame['商品描述'])
feature_names  = vectorizer.get_feature_names()
word_index_dic = vectorizer.vocabulary_
length = (fre_matrix.shape)[1]
newd = {v:k for k,v in word_index_dic.items()}
bag_words = []
tfidf = TfidfTransformer()
tfidf = tfidf.fit_transform(fre_matrix)

for i in range(61497):
    print('----Document %d----' % (i))
    for j in range(length):
        if tfidf[i,j] > 1e-5:
              print( feature_names[j], tfidf[i,j])

In [9]:
n = 0
for label in Counter(frame['标签']).keys():
    if n % 100 == 0:
        print(n)
    choose = frame.loc[frame['标签'] == label].index
    A = fre_matrix[choose].sum(axis=0)
    C = np.zeros((1,length),np.int64)
    index_list = [] 
    for i in range(length):
        if A[0,i] != 0:
            index_list.append(i)
            C[0,i] = len(choose) - A[0,i]
    choose_1 = frame.loc[frame['标签'] != label].index
    B_ = np.zeros((1,length),np.int64)
    for index in choose_1:
        B_ = fre_matrix[index] + B_
    B = np.zeros((1,length),np.int64)
    for index in index_list:
        B[0,index] = B_[0,index]
    D = np.zeros((1,length),np.int64)
    for i in range(length):
        if B[0,i] != 0:
            D[0,i] = len(choose_1) - B[0,i]
    AD_BC_2 = (np.array(A)*np.array(D) - np.array(B)*np.array(C))**2
    chi2 =  AD_BC_2  / ((np.array(A) + np.array(B))*(np.array(C)+ np.array(D)))
    hope_index = {}
    for i in range(length):
        if isnan(chi2[0,i]) == False:
            hope_index[i] = chi2[0,i]
    li = sorted(hope_index.items(),key = lambda x:x[1],reverse = True)
    #for i in range(2):
        #bag_words.append(newd[li[i][0]])
    bag_words.append(newd[li[0][0]])
    n = n + 1

0


c:\python3\lib\site-packages\ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in true_divide


100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200


In [10]:
good_words = list(set(bag_words))
print(len(bag_words),len(set(bag_words)))

2262 2239


In [11]:
def _chi2(matrix,weight):
    matrix_1 = copy.deepcopy(matrix)
    for row in range(matrix_1.shape[0]):
        for word in frame['商品描述'][row].split():
            if word in good_words:
                col = word_index_dic[word]
                matrix_1[row,col] += weight
    return matrix_1
def LR(X,Y):
    X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size = 0.2)
    classifier = LogisticRegression(random_state = 0)
    classifier.fit(X_train, Y_train)# coding:utf-8
    y_pred = classifier.predict(X_test)
    print(np.mean(y_pred == Y_test))
    return (np.mean(y_pred == Y_test))
def RF(X,Y):
    X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size = 0.2)
    classifier = RandomForestClassifier(random_state = 0)
    classifier.fit(X_train, Y_train)# coding:utf-8
    y_pred = classifier.predict(X_test)
    print(np.mean(y_pred == Y_test))
    return (np.mean(y_pred == Y_test))

## 词频加开方

In [12]:
X_matrix = _chi2(fre_matrix,1)
accuracy = LR(X_matrix,frame['标签'])

0.7601268911664226


## 单纯的词频

In [13]:
accuracy = LR(fre_matrix,frame['标签'])

0.7369448511469009


## 单纯的tfidf

In [14]:
accuracy = LR(tfidf,frame['标签'])

0.6394989425736132


## tfidf+开方

In [15]:
X_matrix = _chi2(tfidf,1)
accuracy = LR(X_matrix,frame['标签'])

0.7545143972669595


## tfidf+开方+word2vec每一个维度最大

In [16]:
from gensim.models import word2vec
sentences = word2vec.Text8Corpus('分词拿去做word2vect.txt')
model = word2vec.Word2Vec(sentences,size = 100,window = 900 ,min_count = 3)

c:\python3\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [17]:
X_max = []
Y_max = []
for i in range(len(frame['商品描述'])):
    matrix = []
    misnum = 0
    for word in frame['商品描述'][i].split():
        try:
            matrix.append(model[word].tolist())
        except:
            misnum += 1
    max_list = []
    for j in range(len(matrix[0])):  
        one_list=[]  
        for k in range(len(matrix)):  
            one_list.append(matrix[k][j])  
        max_list.append(max(one_list))
    X_max.append(max_list)
tfidf_chi2 = _chi2(tfidf,1.5)
from scipy.sparse import hstack,vstack,csc_matrix
#word2vec_matrix = csc_matrix(X_max) 可以将X_max转化为稀疏矩阵
tfidf_chi2_word2vec = hstack((tfidf_chi2,X_max))

c:\python3\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [18]:
accuracy = LR(tfidf_chi2_word2vec,frame['标签'])

0.7425573450463641


In [19]:
accuracy = LR(tfidf_chi2,frame['标签'])

0.7586627623230844


## tfidf+开方+word2vec每一个维度均值

In [20]:
corpus = [des.split() for des in frame['商品描述']]

In [21]:
from gensim.models.word2vec import Word2Vec 
model = Word2Vec(size = 100 ,window = 500 , min_count = 3)
model.build_vocab(corpus)
model.train(corpus,total_examples=model.corpus_count,epochs=model.epochs)

(3058934, 3241835)

In [22]:
word2vac_avg = []
for line in range(len(frame['商品描述'])):
    su = np.zeros((1,100))
    words = frame['商品描述'][line].split()
    num = 1
    for word in words:
        try:
            temp = model[word]
        except:
            continue
        else:
            temp = temp.reshape((1,100))
            su += temp
            num = num+1
    if num > 1:
        num -= 1
    su /= num
    word2vac_avg.append(su)

c:\python3\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [23]:
X_svc = []
for i in range(len(word2vac_avg)):
    X_svc.append(list(word2vac_avg[i][0]))

In [24]:
from sklearn.svm import SVC
clf = SVC()
X_train,X_test,Y_train,Y_test = train_test_split(X_svc,frame['标签'],test_size = 0.2)
clf.fit(X_train,Y_train)
result = clf.predict(X_test)
print(np.mean(result == Y_test))

0.24548560273304051


In [25]:
tfidf_chi2 = _chi2(tfidf,1.5)
tfidf_chi2_word2vec_avg = hstack((tfidf_chi2,np.array(word2vac_avg).reshape(-1,100)))

In [26]:
accuracy = LR(tfidf_chi2_word2vec_avg,frame['标签'])

0.755165121197332


### 在试一下啦啦

In [27]:
vectorizer = CountVectorizer(min_df = 3)
fre_matrix = vectorizer.fit_transform(frame['商品描述'])
feature_names  = vectorizer.get_feature_names()
word_index_dic = vectorizer.vocabulary_
length = (fre_matrix.shape)[1]
newd = {v:k for k,v in word_index_dic.items()}
bag_words = []
tfidf = TfidfTransformer()
tfidf = tfidf.fit_transform(fre_matrix)

In [28]:
#X_matrix = min_df_chi2(tfidf,1)
accuracy = LR(tfidf,frame['标签'])

0.6429965836993655


In [29]:
def min_df_chi2(matrix,weight):
    matrix_1 = copy.deepcopy(matrix)
    for row in range(fre_matrix.shape[0]):
        for word in frame['商品描述'][row].split():
            if word in good_words:
                try:
                    col = word_index_dic[word]
                    matrix_1[row,col] += weight
                except:
                    a = 1
    return matrix_1

In [30]:
fre_matrix.shape

(61467, 14379)

## 随机森林

In [31]:
from sklearn.ensemble import RandomForestClassifier
rf_x = _chi2(fre_matrix,1)
RF(rf_x,frame['标签'])

KeyError: '岩田'

In [ ]:
LR(rf_x,frame['标签'])

## 字加词

In [ ]:
from scipy.sparse import hstack,vstack,csc_matrix

vectorizer_char = CountVectorizer(analyzer='char',min_df = 4)
fre_matrix_char = vectorizer_char.fit_transform(frame['商品描述']) # 2783

vectorizer_fre = CountVectorizer(min_df = 4)
fre_matrix_ = vectorizer_fre.fit_transform(frame['商品描述']) #11965

fre_char_matrix = hstack([fre_matrix_char,fre_matrix_])

In [ ]:
acc_1 = LR(fre_matrix_char,frame['标签']) # 按字做
acc_2 = LR(fre_matrix_,frame['标签'])  # 词频
acc_3 = LR(fre_char_matrix,frame['标签'])  #词频加字

In [ ]:
word_index_dic_ = vectorizer_fre.vocabulary_

In [ ]:
matrix_1 = copy.deepcopy(fre_matrix_)
for row in range(fre_matrix_.shape[0]):
    for word in frame['商品描述'][row].split():
        if word in good_words and word in word_index_dic_.keys():
            col = word_index_dic_[word]
            matrix_1[row,col] += 1

In [ ]:
fre_char_matrix = hstack([fre_matrix_char,matrix_1])
acc_4 = LR(fre_char_matrix,frame['标签']) #词加开方 + 字

In [ ]:
LR(matrix_1,frame['标签'])